In [1]:
import pandas as pd 


In [65]:
import numpy as np
np.random.seed(42)
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from tensorflow.keras.models import Model
from  tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate  , BatchNormalization,Dropout
from  tensorflow.keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D,LSTM

from  tensorflow.keras.preprocessing import text, sequence
from  tensorflow.keras.callbacks import Callback
from  tensorflow.keras import regularizers

In [4]:
train=pd.read_csv("train.csv")

In [5]:
len(train)

159571

In [6]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
train.loc[20434]

id                                                35edbce803c6c775
comment_text     How about This\n\nHow about just wait until th...
toxic                                                            0
severe_toxic                                                     0
obscene                                                          0
threat                                                           0
insult                                                           0
identity_hate                                                    0
Name: 20434, dtype: object

In [8]:
test=pd.read_csv("test.csv")

In [8]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [9]:
test_labels=pd.read_csv("test_labels.csv")

In [10]:
test_labels.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [9]:
submission = pd.read_csv('sample_submission.csv')

In [12]:
submission .head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5


In [10]:
X_train = train["comment_text"].fillna("fillna").values

In [11]:
 set(train["comment_text"].isna())

{False}

In [12]:
 train["comment_text"]

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
5         "\n\nCongratulations from me as well, use the ...
6              COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
7         Your vandalism to the Matt Shirvington article...
8         Sorry if the word 'nonsense' was offensive to ...
9         alignment on this subject and which are contra...
10        "\nFair use rationale for Image:Wonju.jpg\n\nT...
11        bbq \n\nbe a man and lets discuss it-maybe ove...
12        Hey... what is it..\n@ | talk .\nWhat is it......
13        Before you start throwing accusations and warn...
14        Oh, and the girl above started her arguments w...
15        "\n\nJuelz Santanas Age\n\nIn 2002, Juelz Sant...
16        Bye! \n\nDon't look, come or t

In [13]:
 train["comment_text"].fillna("fillna").values

array(["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
       ...,
       'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.',
       'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.',
       '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of communit

In [14]:
X_train.shape

(159571,)

In [15]:
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [16]:
y_train

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [17]:
X_test = test["comment_text"].fillna("fillna").values

In [18]:
X_test.shape

(153164,)

In [19]:

max_features = 30000
maxlen = 100
embed_size = 300


In [20]:
tokenizer = text.Tokenizer(num_words=max_features)

In [21]:
list(X_train) 

["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
 "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
 "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
 '"\nMore\nI can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on r

In [22]:
tokenizer.fit_on_texts(list(X_train) + list(X_test))

In [23]:
tokenizer.word_index

{'the': 1,
 'to': 2,
 'of': 3,
 'a': 4,
 'and': 5,
 'you': 6,
 'i': 7,
 'is': 8,
 'that': 9,
 'in': 10,
 'it': 11,
 'for': 12,
 'this': 13,
 'not': 14,
 'on': 15,
 'be': 16,
 'as': 17,
 'are': 18,
 'have': 19,
 'your': 20,
 'with': 21,
 'if': 22,
 'article': 23,
 'was': 24,
 'or': 25,
 'but': 26,
 'an': 27,
 'wikipedia': 28,
 'page': 29,
 'my': 30,
 'from': 31,
 'by': 32,
 'at': 33,
 'do': 34,
 'about': 35,
 'so': 36,
 'me': 37,
 'what': 38,
 'can': 39,
 'all': 40,
 'there': 41,
 'has': 42,
 'would': 43,
 'no': 44,
 'will': 45,
 'talk': 46,
 'one': 47,
 'like': 48,
 'just': 49,
 'please': 50,
 'they': 51,
 'he': 52,
 'we': 53,
 'which': 54,
 'any': 55,
 'should': 56,
 'been': 57,
 'more': 58,
 "don't": 59,
 'some': 60,
 'other': 61,
 'who': 62,
 'see': 63,
 'here': 64,
 'think': 65,
 'also': 66,
 'fuck': 67,
 'his': 68,
 'because': 69,
 'know': 70,
 "it's": 71,
 "i'm": 72,
 'up': 73,
 'how': 74,
 'people': 75,
 'out': 76,
 'only': 77,
 'why': 78,
 'edit': 79,
 'when': 80,
 'am': 81,
 '

In [24]:
tokenizer.word_index

{'the': 1,
 'to': 2,
 'of': 3,
 'a': 4,
 'and': 5,
 'you': 6,
 'i': 7,
 'is': 8,
 'that': 9,
 'in': 10,
 'it': 11,
 'for': 12,
 'this': 13,
 'not': 14,
 'on': 15,
 'be': 16,
 'as': 17,
 'are': 18,
 'have': 19,
 'your': 20,
 'with': 21,
 'if': 22,
 'article': 23,
 'was': 24,
 'or': 25,
 'but': 26,
 'an': 27,
 'wikipedia': 28,
 'page': 29,
 'my': 30,
 'from': 31,
 'by': 32,
 'at': 33,
 'do': 34,
 'about': 35,
 'so': 36,
 'me': 37,
 'what': 38,
 'can': 39,
 'all': 40,
 'there': 41,
 'has': 42,
 'would': 43,
 'no': 44,
 'will': 45,
 'talk': 46,
 'one': 47,
 'like': 48,
 'just': 49,
 'please': 50,
 'they': 51,
 'he': 52,
 'we': 53,
 'which': 54,
 'any': 55,
 'should': 56,
 'been': 57,
 'more': 58,
 "don't": 59,
 'some': 60,
 'other': 61,
 'who': 62,
 'see': 63,
 'here': 64,
 'think': 65,
 'also': 66,
 'fuck': 67,
 'his': 68,
 'because': 69,
 'know': 70,
 "it's": 71,
 "i'm": 72,
 'up': 73,
 'how': 74,
 'people': 75,
 'out': 76,
 'only': 77,
 'why': 78,
 'edit': 79,
 'when': 80,
 'am': 81,
 '

In [25]:
list(tokenizer.word_index)[-1]

'angeln'

In [26]:
tokenizer.word_index["angeln"]

394787

In [27]:
len(tokenizer.word_index)

394787

In [28]:
max_id = len(tokenizer.word_index)
max_id

394787

In [29]:
tokenizer.document_count  # whole corpus 

312735

In [30]:
X_train = tokenizer.texts_to_sequences(X_train)

In [31]:
max(X_train )  # 30000 以内

[29996,
 120,
 178,
 1282,
 22,
 6,
 103,
 498,
 1,
 120,
 337,
 25,
 684,
 12,
 1,
 29996,
 6,
 670,
 15,
 180,
 665,
 647,
 769]

In [32]:
X_test = tokenizer.texts_to_sequences(X_test)

In [33]:
len(X_train)

159571

In [34]:
X_train[0]

[733,
 78,
 1,
 140,
 131,
 182,
 30,
 712,
 4438,
 10284,
 1252,
 86,
 368,
 51,
 2230,
 14039,
 49,
 6744,
 15,
 60,
 2624,
 151,
 7,
 2832,
 33,
 115,
 1246,
 16129,
 2517,
 5,
 50,
 59,
 256,
 1,
 370,
 31,
 1,
 46,
 29,
 144,
 72,
 3931,
 89,
 4208,
 6368,
 2687,
 1183]

In [35]:
len(X_train[0])

47

In [37]:
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [ ]:
x_train.shape

In [ ]:
X_train[0]

In [ ]:
x_train[0]

In [ ]:
EMBEDDING_FILE = 'crawl-300d-2M.vec'

In [ ]:

def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [ ]:
embeddings_index

In [38]:
import joblib

In [ ]:
joblib.dump(embeddings_index,"embeddings_index.pkl")

In [ ]:
len(embeddings_index['the'])

In [39]:
embeddings_index =joblib.load("embeddings_index.pkl")

In [40]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))

In [41]:
word_index 

{'the': 1,
 'to': 2,
 'of': 3,
 'a': 4,
 'and': 5,
 'you': 6,
 'i': 7,
 'is': 8,
 'that': 9,
 'in': 10,
 'it': 11,
 'for': 12,
 'this': 13,
 'not': 14,
 'on': 15,
 'be': 16,
 'as': 17,
 'are': 18,
 'have': 19,
 'your': 20,
 'with': 21,
 'if': 22,
 'article': 23,
 'was': 24,
 'or': 25,
 'but': 26,
 'an': 27,
 'wikipedia': 28,
 'page': 29,
 'my': 30,
 'from': 31,
 'by': 32,
 'at': 33,
 'do': 34,
 'about': 35,
 'so': 36,
 'me': 37,
 'what': 38,
 'can': 39,
 'all': 40,
 'there': 41,
 'has': 42,
 'would': 43,
 'no': 44,
 'will': 45,
 'talk': 46,
 'one': 47,
 'like': 48,
 'just': 49,
 'please': 50,
 'they': 51,
 'he': 52,
 'we': 53,
 'which': 54,
 'any': 55,
 'should': 56,
 'been': 57,
 'more': 58,
 "don't": 59,
 'some': 60,
 'other': 61,
 'who': 62,
 'see': 63,
 'here': 64,
 'think': 65,
 'also': 66,
 'fuck': 67,
 'his': 68,
 'because': 69,
 'know': 70,
 "it's": 71,
 "i'm": 72,
 'up': 73,
 'how': 74,
 'people': 75,
 'out': 76,
 'only': 77,
 'why': 78,
 'edit': 79,
 'when': 80,
 'am': 81,
 '

In [42]:
nb_words

30000

In [43]:
embedding_matrix = np.zeros((nb_words, embed_size))
embedding_matrix .shape

(30000, 300)

In [ ]:
word_index.items()

In [44]:

for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [45]:
embedding_matrix.shape

(30000, 300)

In [ ]:
embedding_matrix[0]

In [ ]:
embedding_matrix[1]==embeddings_index['the']

In [46]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [47]:
import warnings
warnings.filterwarnings('ignore')

In [48]:
import os
os.environ['OMP_NUM_THREADS'] = '4'

In [ ]:
x_train.shape

In [ ]:

def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(80, return_sequences=True))(x)   #160
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    print(outp.shape)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [ ]:
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95, random_state=233)

In [ ]:
 y_tra.shape

In [ ]:
X_tra.shape

In [ ]:
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(X_tra)

In [ ]:
x.shape

In [ ]:
x = SpatialDropout1D(0.2)(x)

In [ ]:
x.shape

In [ ]:
x[:3,:,:].shape

In [ ]:
 x = Bidirectional(GRU(80, return_sequences=True))(x[:3,:,:])

In [ ]:
x.shape

In [ ]:
avg_pool = GlobalAveragePooling1D()(x)

In [ ]:
avg_pool.shape

In [ ]:
 max_pool = GlobalMaxPooling1D()(x)

In [ ]:
 max_pool .shape

In [ ]:
conc = concatenate([avg_pool, max_pool])
conc.shape

In [ ]:
outp = Dense(6, activation="sigmoid")(conc)
outp.shape

In [ ]:
model = get_model()

In [ ]:
model.summary()

In [ ]:
batch_size = 32
epochs = 2

In [ ]:
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)


In [58]:

def get_my_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.25)(x)
    x=   BatchNormalization()(x)  
    x = Bidirectional(LSTM(100, return_sequences=True))(x)   #160
    x = SpatialDropout1D(0.25)(x)
    x=   BatchNormalization()(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    
    x = Dense(30, activation="relu")( conc)
    #print(x.shape)
    x = Dropout(0.25)(x)
    x=   BatchNormalization()(  x)  

    outp = Dense(6, activation="sigmoid")(x)
    #print(outp.shape)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [59]:
model2 = get_my_model()

In [52]:
X_tra.shape

(151592, 100)

In [53]:
 y_tra.shape

(151592, 6)

In [55]:
batch_size = 32
epochs = 2

In [60]:
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)


In [61]:

hist = model2.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)


Train on 151592 samples, validate on 7979 samples
Epoch 1/2
151584/151592 [============================>.] - ETA: 0s - loss: 0.0864 - accuracy: 0.9684
 ROC-AUC - epoch: 1 - score: 0.975207 

151592/151592 [==============================] - 1994s 13ms/sample - loss: 0.0864 - accuracy: 0.9684 - val_loss: 0.0546 - val_accuracy: 0.9812
Epoch 2/2
151584/151592 [============================>.] - ETA: 0s - loss: 0.0532 - accuracy: 0.9807
 ROC-AUC - epoch: 2 - score: 0.976982 

151592/151592 [==============================] - 1902s 13ms/sample - loss: 0.0532 - accuracy: 0.9807 - val_loss: 0.0519 - val_accuracy: 0.9816


In [63]:
y_pred = model2.predict(x_test, batch_size=1024)
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('submission.csv', index=False)

In [66]:

def get_my_model3():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.3)(x)
    x=   BatchNormalization()(x)  
    
    x = Bidirectional(LSTM(80, return_sequences=True,kernel_regularizer=regularizers.l2(0.01)))(x)   #160
    x = SpatialDropout1D(0.3)(x)
    x=   BatchNormalization()(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])

    outp = Dense(6, activation="sigmoid",kernel_regularizer=regularizers.l2(0.01))(conc)
    #print(outp.shape)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [67]:
model3=get_my_model3()

In [ ]:

hist = model3.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)


Train on 151592 samples, validate on 7979 samples
Epoch 1/2
151584/151592 [============================>.] - ETA: 0s - loss: 0.1550 - accuracy: 0.9788
 ROC-AUC - epoch: 1 - score: 0.972672 

151592/151592 [==============================] - 1740s 11ms/sample - loss: 0.1550 - accuracy: 0.9788 - val_loss: 0.0830 - val_accuracy: 0.9805
Epoch 2/2
149568/151592 [============================>.] - ETA: 22s - loss: 0.0823 - accuracy: 0.9809

In [ ]:
#woc 加上正则化，看到正确率开始就上去得很快 ，